In [154]:
import re
import json
import nltk
import plotly
import pandas as pd
import numpy as np

from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk import pos_tag, word_tokenize

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.externals import joblib

from flask import Flask, render_template, request, jsonify, redirect
from flask_sqlalchemy import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect


class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)


def tokenize(text):
    # replace the webpage with a urlplaceholder
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    # tokenize text
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    # lemmatize, turn into lowercase, strip spaces
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens


# load the data.
engine = create_engine("sqlite:///./PY/DisasterResponse.db", echo=True)
session = Session(engine)

In [155]:
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('disaster_response', engine)

In [156]:
model = joblib.load("classifier.pkl")

In [12]:
foo = results[0]
foobar = np.where(foo == 1)[0]
bar = [column_names[x] for x in foobar]
bar

NameError: name 'results' is not defined

In [157]:
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /Users/knishina/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/knishina/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/knishina/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [153]:
foobar = np.where(classification_labels == 1)
foobar

(array([0]),)

In [168]:
text = tokenize("earthquake")
text

['earthquake']

In [161]:
classification_labels = model.predict(text)[0]
classification_labels

array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0])

In [162]:
classification_results = dict(zip(df.columns[4:], classification_labels))
classification_results

{'related': 1,
 'request': 0,
 'offer': 0,
 'aid_related': 0,
 'medical_help': 0,
 'medical_products': 0,
 'search_and_rescue': 0,
 'security': 0,
 'military': 0,
 'child_alone': 0,
 'water': 0,
 'food': 0,
 'shelter': 0,
 'clothing': 0,
 'money': 0,
 'missing_people': 0,
 'refugees': 0,
 'death': 0,
 'other_aid': 0,
 'infrastructure_related': 0,
 'transport': 0,
 'buildings': 0,
 'electricity': 0,
 'tools': 0,
 'hospitals': 0,
 'shops': 0,
 'aid_centers': 0,
 'other_infrastructure': 0,
 'weather_related': 1,
 'floods': 0,
 'storm': 0,
 'fire': 0,
 'earthquake': 1,
 'cold': 0,
 'other_weather': 0,
 'direct_report': 0}

In [163]:
listy = []
for classification in classification_results:
    if classification_results[classification] == 1:
        listy.append(classification)

In [164]:
listy

['related', 'weather_related', 'earthquake']

In [165]:
lister = {}
for thing in range(len(listy)):
    lister[thing] = listy[thing]

In [166]:
lister

{0: 'related', 1: 'weather_related', 2: 'earthquake'}

In [171]:
query = tokenize("earthquake")
classification_labels = model.predict(query)[0]
classification_results = dict(zip(df.columns[4:], classification_labels))

listy = []
for classification in classification_results:
    if classification_results[classification] == 1:
        listy.append(classification)
lister = {}
for thing in range(len(listy)):
    lister[thing] = listy[thing]